In [1]:
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time
import threading
import os

service = Service(r'C:\Users\Sercan\Downloads\geckodriver-v0.35.0-win32\geckodriver.exe')
options = Options()
options.add_argument("--headless")
options.binary_location = r'C:\Program Files\Mozilla Firefox\firefox.exe'
driver = webdriver.Firefox(service=service, options=options)

base_url = "https://marketplace.skymavis.com/collections/lumiterra?auction=Sale&page={}&type=consumable&type=material&type=taskticket"
file_path = r'C:\Users\Sercan\dist\nft_market_material_prices_sorted.xlsx'

def update_data():
    while not stop_flag.is_set():
        new_data = pd.DataFrame(columns=['Item Name', 'Price'])
        
        for page in range(1, 7): 
            url = base_url.format(page)
            driver.get(url)
            
            try:
                element_present = EC.presence_of_element_located((By.XPATH, "//div[contains(@class, 'Card_Footer__vcM3_')]"))
                WebDriverWait(driver, 5).until(element_present)
                
                items = driver.find_elements(By.XPATH, "//div[contains(@class, 'Card_Footer__vcM3_')]")
                
                if not items:
                    print(f"Sayfada öğe bulunamadı: {url}")
                    continue
                
                for item in items:
                    try:
                        item_name_elements = item.find_elements(By.XPATH, ".//div[contains(@class, 'Erc1155Card_name__Z9fOp')]")
                        item_name = item_name_elements[-1].text.strip() if item_name_elements else 'N/A'
                        
                        price_element = item.find_element(By.XPATH, ".//div[contains(@class, 'Erc1155Card_prices__kLn2L')]//h5[contains(@class, 'TokenPrice_price__ZrJBZ')]")
                        price = price_element.text.strip() if price_element else 'N/A'
                        
                        print(f"Item: {item_name}, Price: {price}")
                        
                        new_data = pd.concat([new_data, pd.DataFrame([{'Item Name': item_name, 'Price': price}])], ignore_index=True)
                    except Exception as e:
                        print(f"Öğe verileri alınırken bir hata oluştu: {e}")
                        continue
            
            except Exception as e:
                print(f"Bir hata oluştu: {e}")
                continue
        
        new_data['Price'] = pd.to_numeric(new_data['Price'], errors='coerce')
        new_data.sort_values(by='Item Name', inplace=True)
        
        try:
            if os.path.exists(file_path):
                existing_data = pd.read_excel(file_path)
                updated_data = pd.concat([existing_data, new_data])
                updated_data = updated_data.groupby('Item Name').agg({'Price': 'last'}).reset_index()
            else:
                updated_data = new_data.groupby('Item Name').agg({'Price': 'last'}).reset_index()
            
            updated_data.to_excel(file_path, index=False)
            print("Excel dosyası başarıyla güncellendi.")
        except Exception as e:
            print(f"Excel dosyasına yazarken bir hata oluştu: {e}")
        
        time.sleep(5)

stop_flag = threading.Event()

def wait_for_exit():
    input("Verileri güncellemeye başladık. Durdurmak için Enter'a basın...")
    stop_flag.set()
    print("İşlem durduruluyor...")

threading.Thread(target=update_data, daemon=True).start()
wait_for_exit()

driver.quit()


Item: Empty Bottle, Price: 0.075
Item: Cotton Seed, Price: 0.088
Item: Cactus Seed, Price: 0.089
Item: Pigmy Sundew Seed, Price: 0.094
Item: Garlic Seed, Price: 0.098
Item: Peas Seed, Price: 0.10
Item: Carrot Seed, Price: 0.10
Item: Hard-shelled Fruit Seed, Price: 0.10
Item: Wheat Seed, Price: 0.10
Item: Explosive Hot Pepper Seed, Price: 0.10
Item: Blue Mushroom Seed, Price: 0.11
Item: Grape Seed, Price: 0.11
Item: Sunflower Seed, Price: 0.11
Item: Lv 1 Packaging Paper, Price: 0.12
Item: Potato Seed, Price: 0.12
Item: Bamboo Seed, Price: 0.12
Item: Trifolium Seed, Price: 0.12
Item: Lv 4 Packaging Paper, Price: 0.14
Item: Farm0-3_20240811, Price: 0.14
Item: Farm0-3_20240813, Price: 0.15
Item: Lv 2 Packaging Paper, Price: 0.17
Item: Ordinary Wood, Price: 0.19
Item: Chomper Seed, Price: 0.19
Item: Combat Material Lottery 1-6, Price: 0.20
Item: Combat Material 0-3_20240810, Price: 0.20
Item: Medal of Vitality, Price: 0.20
Item: Combat0-3_20240814, Price: 0.21
Item: Combat0-3_20240811, Pric

Verileri güncellemeye başladık. Durdurmak için Enter'a basın... 


İşlem durduruluyor...
Item: Farm Material 4-6_20240814, Price: 1.80
Item: Lord Fang, Price: 1.90
Item: Lv 1 Gather Essence, Price: 1.99
Item: Elven Wings, Price: 2.00
Item: Dark Spice, Price: 2.00
Item: Lv 1 Combat Essence, Price: 2.00
Item: Lv 3 Energy Restoration Potion 2.0, Price: 2.15
Item: Lv 6 Packaging Paper, Price: 2.20
Item: Junior Farming Admission Ticket, Price: 2.32
Item: Junior Gather Admission Ticket, Price: 2.39
Item: Lv 1 Agricultural Livestock Essence, Price: 2.40
Item: Lv 6 Packaging box, Price: 2.40
Item: Lv 8 Packaging Paper, Price: 2.50
Item: Smelting Fruit Seed, Price: 2.50
Item: Chomper, Price: 2.50
Item: Lv 7 Packaging Paper, Price: 2.66
Item: Flower Peak Admission Ticket, Price: 2.68
Item: Lv 1 Agricultural Planting Essence, Price: 2.70
Item: Lv 4 Energy Restoration Potion 2.0, Price: 2.86
Item: Elf Snowball, Price: 3.00
Item: Cassava, Price: 3.00
Item: Combat Material 4-6_20240815, Price: 3.00
Item: Refined Arrow, Price: 3.30
Item: Luminescent Bone Powder, Pri

Exception in thread Thread-5 (update_data):
Traceback (most recent call last):
  File "C:\Users\Sercan\anaconda3\Lib\threading.py", line 1073, in _bootstrap_inner
    self.run()
  File "C:\Users\Sercan\anaconda3\Lib\threading.py", line 1010, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\Sercan\AppData\Local\Temp\ipykernel_15768\889013390.py", line 27, in update_data
  File "C:\Users\Sercan\anaconda3\Lib\site-packages\selenium\webdriver\remote\webdriver.py", line 363, in get
    self.execute(Command.GET, {"url": url})
  File "C:\Users\Sercan\anaconda3\Lib\site-packages\selenium\webdriver\remote\webdriver.py", line 354, in execute
    self.error_handler.check_response(response)
  File "C:\Users\Sercan\anaconda3\Lib\site-packages\selenium\webdriver\remote\errorhandler.py", line 229, in check_response
    raise exception_class(message, screen, stacktrace)
selenium.common.exceptions.InvalidSessionIdException: Message: Tried to run command without establishing a conne